<a href="https://colab.research.google.com/github/2359181042/GNN_cs224w/blob/main/Wells_GNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric
!pip install pytorch_lightning

     |████████████████████████████████| 2.6 MB 2.1 MB/s 
     |████████████████████████████████| 1.4 MB 1.2 MB/s 
     |████████████████████████████████| 222 kB 5.2 MB/s 
     |████████████████████████████████| 376 kB 9.2 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
     |████████████████████████████████| 813 kB 4.9 MB/s 
     |████████████████████████████████| 10.6 MB 44.7 MB/s 
     |████████████████████████████████| 234 kB 45.5 MB/s 
     |████████████████████████████████| 636 kB 26.5 MB/s 
     |████████████████████████████████| 118 kB 66.9 MB/s 
     |████████████████████████████████| 829 kB 45.8 MB/s 
     |████████████████████████████████| 1.3 MB 48.5 MB/s 
     |████████████████████████████████| 142 kB 71.5 MB/s 
     |████████████████████████████████| 294 kB 40.7 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=9b20f556677ba468022f756f400f01ef2e79f889b27a52de4a342be0437e6888
  Stored in directory: /root/.cache/pip/wh

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import os
import json
import numpy as np
import pandas as pd
DATA_PATH = 'gdrive/My Drive/DaguRiver/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/gdrive


In [3]:
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import math
import matplotlib

import pandas as pd
import numpy as np
from tqdm import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid',palette='muted',font_scale = 1.2)
HAPPY_COLORS_PALETTE = ['#01BEFE','#FFDD00','#FF7D00','#FF006D','#ADFF02','#8F00FF']
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize']=12,8
tqdm.pandas()


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# 1. Data Processing
## Totally 10 features: salinity, level, pump, infil, infil_s, rain,loc, duration, well_conn, well_weight.
## sequence length is 18, features salinity-rain is based on 11 opration wells, while feaures loc is based on 1 observe well. The well_conn and well weight are calculated based on the relative distance between two wells.

## 1.1 Prepare data

In [4]:
test1=pd.read_csv(os.path.join(DATA_PATH,'df_of_well.csv'))
salinity=[]
level=[]
pump=[]
infil=[]
infil_s=[]
rain=[]
for i in range(21):
    salinity.append(np.array(test1.iloc[:,0+i*6]))

    level.append(np.array(test1.iloc[:,1+i*6]))

    pump.append(np.array(test1.iloc[:,2+i*6]))

    infil.append(np.array(test1.iloc[:,3+i*6]))

    infil_s.append(np.array(test1.iloc[:,4+i*6]))

    rain.append(np.array(test1.iloc[:,5+i*6]))
salinity=np.array(salinity)
level=np.array(level)
pump = np.array(pump)
infil = np.array(infil)
infil_s = np.array(infil_s)
rain=np.array(rain)            

In [7]:
df3=pd.read_csv(os.path.join(DATA_PATH,'date_info.csv'))
df3['Date'] = pd.to_datetime(df3['Date'])

df3['Day'] = df3['Date'].dt.day
df3['Month']=df3['Date'].dt.month


df3['Duration'] = df3['Day']-17 + 30*(df3['Month']-9) 
duration = df3['Duration'].to_numpy()

[ 1  1  1  1  1  1  1  1  2  2  2  2  2  3  3  3  3  3  4  4  4  4  4  4
  5  5  5  5  6  6  6  7  7  7  7  8  8  8  9  9  9  9 10 10 10 11 11 11
 12 12 12 13 13 13 13 14 14 14 14 15 15 15 15 16 16 16 17 17 17 18 18 18
 19 19 19 20 20 20 21 21 21]


In [8]:
Well_location= pd.read_csv(os.path.join(DATA_PATH,'sets.csv'))

NE=[]
North=[]
East=[]
for i in range(1,22):
    N=Well_location.iloc[0][i]
    North.append(N)
    E=Well_location.iloc[1][i]
    East.append(E)
    NE.append((N,E))

In [9]:
from math import sin, cos, sqrt, atan2, radians
R = 6378.1370
def get_conn_attr(N,E):
    Distance = []
    con_i=[]
    con_j=[]
    for i in range(len(N)):
        for j in range(len(N)):
                lat1 = radians(N[i])
                lon1 = radians(E[i])
                lat2 = radians(N[j])
                lon2 = radians(E[j])
                dlon = lon2 - lon1
                dlat = lat2 - lat1
                a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
                c = 2 * atan2(sqrt(a), sqrt(1 - a))
                distance = R * c
                if (distance < 0.1) & (distance > 0):
                    con_i.append(i)
                    con_j.append(j)
                    Distance.append(distance)

    conn=torch.tensor([con_i,con_j])
    edge_attr = torch.tensor([Distance])
    edge_attr = edge_attr.transpose(0, 1) 
    edge_attr = edge_attr.unsqueeze(0).repeat(18,1,1)
    return conn, edge_attr

## 1.2 Create dataset\

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.utils.data import Dataset, DataLoader
import os
import json
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

TIME_WINDOW = 18
PRED_TIME = 18
##well o 2 4 5  6  8 10 for train_set, 3 9 for validation set, 7 11 for test set
## North[]. East[] 21 用于传入get_conn_attr 计算不同的图数据
 

def create_dataset(salinity,level,pump,infil,infil_s,rain,NE,duration):
    index=0
    train_sequences=[]
    val_sequences=[]
    test_sequences=[]
#O2
    while (index <= (81-PRED_TIME-TIME_WINDOW)):

        s=salinity[11:21,index:index+TIME_WINDOW]
        l=level[11:21,index:index+TIME_WINDOW]
        p=pump[11:21,index:index+TIME_WINDOW]
        i=infil[11:21,index:index+TIME_WINDOW]
        i_s=infil_s[11:21,index:index+TIME_WINDOW]
        r = rain[11:21,index:index+TIME_WINDOW]
        N0= North[11:21]+[North[0]]
        E0= East[11:21]+[East[0]]
        conn,edge_attr = get_conn_attr(N0,E0)
        
        loc = NE[0]
        t=duration[index]
        y = salinity[0,index+TIME_WINDOW:index+TIME_WINDOW+PRED_TIME]
        
        
        
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1
    while ((81-PRED_TIME-TIME_WINDOW)<index<81):
        index +=1

        
#O3        
    while ( 81<=index<=(2*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-1*81:index+TIME_WINDOW-1*81]
        l=level[11:21,index-1*81:index+TIME_WINDOW-1*81]
        p=pump[11:21,index-1*81:index+TIME_WINDOW-1*81]
        i=infil[11:21,index-1*81:index+TIME_WINDOW-1*81]
        i_s=infil_s[11:21,index-1*81:index+TIME_WINDOW-1*81]
        r = rain[11:21,index-1*81:index+TIME_WINDOW-1*81]
        N1= North[11:21]+[North[1]]
        E1= East[11:21]+[East[1]]
        conn,edge_attr = get_conn_attr(N1,E1)
        loc = NE[1]
        t=duration[index-81]
        y = salinity[1,index-1*81+TIME_WINDOW:index-1*81+TIME_WINDOW+PRED_TIME]
        val_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((2*81-PRED_TIME-TIME_WINDOW)<index<2*81):
        index +=1    

#O4
    while ( 2*81<=index<=(3*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-2*81:index+TIME_WINDOW-2*81]
        l=level[11:21,index-2*81:index+TIME_WINDOW-2*81]
        p=pump[11:21,index-2*81:index+TIME_WINDOW-2*81]
        i=infil[11:21,index-2*81:index+TIME_WINDOW-2*81]
        i_s=infil_s[11:21,index-2*81:index+TIME_WINDOW-2*81]
        r = rain[11:21,index-2*81:index+TIME_WINDOW-2*81]
        N2= North[11:21]+[North[2]]
        E2= East[11:21]+[East[2]]
        conn,edge_attr = get_conn_attr(N2,E2)
        loc = NE[2]
        t=duration[index-81*2]
        y = salinity[2,index-2*81+TIME_WINDOW:index-2*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((3*81-PRED_TIME-TIME_WINDOW)<index<3*81):
        index +=1     

#O5 print('O4finish')      
    while ( 3*81<=index<=(4*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-3*81:index-3*81+TIME_WINDOW]
        l=level[11:21,index-3*81:index-3*81+TIME_WINDOW]
        p=pump[11:21,index-3*81:index-3*81+TIME_WINDOW]
        i=infil[11:21,index-3*81:index-3*81+TIME_WINDOW]
        i_s=infil_s[11:21,index-3*81:index-3*81+TIME_WINDOW]
        r = rain[11:21,index-3*81:index-3*81+TIME_WINDOW]
        N3= North[11:21]+[North[3]]
        E3= East[11:21]+[East[3]]
        conn,edge_attr = get_conn_attr(N3,E3)
        loc = NE[3]
        t=duration[index-81*3]
        y = salinity[3,index-3*81+TIME_WINDOW:index-3*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((4*81-PRED_TIME-TIME_WINDOW)<index<4*81):
        index +=1           
#O6       
    while ( 4*81<=index<=(5*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-4*81:index-4*81+TIME_WINDOW]
        l=level[11:21,index-4*81:index-4*81+TIME_WINDOW]
        p=pump[11:21,index-4*81:index-4*81+TIME_WINDOW]
        i=infil[11:21,index-4*81:index-4*81+TIME_WINDOW]
        i_s=infil_s[11:21,index-4*81:index-4*81+TIME_WINDOW]
        r = rain[11:21,index-4*81:index-4*81+TIME_WINDOW]
        N4= North[11:21]+[North[4]]
        E4= East[11:21]+[East[4]]
        conn,edge_attr = get_conn_attr(N4,E4)
        loc = NE[4]
        t=duration[index-81*4]
        y = salinity[4,index-4*81+TIME_WINDOW:index-4*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((5*81-PRED_TIME-TIME_WINDOW)<index<5*81):
        index +=1  

#O7       
    while ( 5*81<=index<=(6*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-5*81:index-5*81+TIME_WINDOW]
        l=level[11:21,index-5*81:index-5*81+TIME_WINDOW]
        p=pump[11:21,index-5*81:index-5*81+TIME_WINDOW]
        i=infil[11:21,index-5*81:index-5*81+TIME_WINDOW]
        i_s=infil_s[11:21,index-5*81:index-5*81+TIME_WINDOW]
        r = rain[11:21,index-5*81:index-5*81+TIME_WINDOW]
        N5= North[11:21]+[North[5]]
        E5= East[11:21]+[East[5]]
        conn,edge_attr = get_conn_attr(N5,E5)
        loc = NE[5]
        t=duration[index-81*5]
        y = salinity[5,index-5*81+TIME_WINDOW:index-5*81+TIME_WINDOW+PRED_TIME]
        test_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((6*81-PRED_TIME-TIME_WINDOW)<index<6*81):
        index +=1 
#O8      
    while ( 6*81<=index<=(7*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-6*81:index-6*81+TIME_WINDOW]
        l=level[11:21,index-6*81:index-6*81+TIME_WINDOW]
        p=pump[11:21,index-6*81:index-6*81+TIME_WINDOW]
        i=infil[11:21,index-6*81:index-6*81+TIME_WINDOW]
        i_s=infil_s[11:21,index-6*81:index-6*81+TIME_WINDOW]
        r = rain[11:21,index-6*81:index-6*81+TIME_WINDOW]
        N6= North[11:21]+[North[6]]
        E6= East[11:21]+[East[6]]
        conn,edge_attr = get_conn_attr(N6,E6)
        loc = NE[6]
        t=duration[index-81*6]
        y = salinity[6,index-6*81+TIME_WINDOW:index-6*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((7*81-PRED_TIME-TIME_WINDOW)<index<7*81):
        index +=1         
#O9        
    while ( 7*81<=index<=(8*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-7*81:index-7*81+TIME_WINDOW]
        l=level[11:21,index-7*81:index-7*81+TIME_WINDOW]
        p=pump[11:21,index-7*81:index-7*81+TIME_WINDOW]
        i=infil[11:21,index-7*81:index-7*81+TIME_WINDOW]
        i_s=infil_s[11:21,index-7*81:index-7*81+TIME_WINDOW]
        r = rain[11:21,index-7*81:index-7*81+TIME_WINDOW]
        N7= North[11:21]+[North[7]]
        E7= East[11:21]+[East[7]]
        conn,edge_attr = get_conn_attr(N7,E7)
        loc = NE[7]
        t=duration[index-81*7]
        y = salinity[7,index-7*81+TIME_WINDOW:index-7*81+TIME_WINDOW+PRED_TIME]
        val_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1

    while ((8*81-PRED_TIME-TIME_WINDOW)<index<8*81):
        index +=1 
#O10        
    while ( 8*81<=index<=(9*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-8*81:index-8*81+TIME_WINDOW]
        l=level[11:21,index-8*81:index-8*81+TIME_WINDOW]
        p=pump[11:21,index-8*81:index-8*81+TIME_WINDOW]
        i=infil[11:21,index-8*81:index-8*81+TIME_WINDOW]
        i_s=infil_s[11:21,index-8*81:index-8*81+TIME_WINDOW]
        r = rain[11:21,index-8*81:index-8*81+TIME_WINDOW]
        N8= North[11:21]+[North[8]]
        E8= East[11:21]+[East[8]]
        conn,edge_attr = get_conn_attr(N8,E8)
        loc = NE[8]
        t=duration[index-81*8]
        y = salinity[8,index-8*81+TIME_WINDOW:index-8*81+TIME_WINDOW+PRED_TIME]
        train_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1
    while ((9*81-PRED_TIME-TIME_WINDOW)<index<9*81):
        index +=1 
        
#O11        
    while ( 9*81<=index<=(10*81-PRED_TIME-TIME_WINDOW)):
        s=salinity[11:21,index-9*81:index-9*81+TIME_WINDOW]
        l=level[11:21,index-9*81:index-9*81+TIME_WINDOW]
        p=pump[11:21,index-9*81:index-9*81+TIME_WINDOW]
        i=infil[11:21,index-9*81:index-9*81+TIME_WINDOW]
        i_s=infil_s[11:21,index-9*81:index-9*81+TIME_WINDOW]
        r = rain[11:21,index-9*81:index-9*81+TIME_WINDOW]
        N9= North[11:21]+[North[9]]
        E9= East[11:21]+[East[9]]
        conn,edge_attr = get_conn_attr(N9,E9)
        loc = NE[9]
        t=duration[index-81*9]
        y = salinity[9,index-9*81+TIME_WINDOW:index-9*81+TIME_WINDOW+PRED_TIME]
        test_sequences.append((s,l,p,i,i_s,r,loc,t,conn,edge_attr,y))
        index+=1
    while ((10*81-PRED_TIME-TIME_WINDOW)<index<10*81):
        index +=1 


        
        
    return train_sequences,val_sequences,test_sequences

In [13]:
train_sequences,val_sequences, test_sequences = create_dataset(salinity,level,pump,infil,infil_s,rain,NE,duration)
print(len(train_sequences))
print(len(test_sequences))
print(len(val_sequences))
print(train_sequences[92][9].shape)

276
92
92
torch.Size([18, 22, 1])


## 1.3 Create datamodule and dataloader

In [14]:
class WellDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self,index):
        sal,lev,pum,inf,inf_s,rai,loca,dur,conn,edge_attr, pred_y = self.sequences[index]
        return dict(
            sal_sequence = torch.tensor(sal),
            lev_sequence = torch.tensor(lev),
            pum_sequence = torch.tensor(pum),
            inf_sequence = torch.tensor(inf),
            inf_s_sequence = torch.tensor(inf_s),
            rai_sequence = torch.tensor(rai),
            loc_sequence = torch.tensor(loca),
            dur_sequence = torch.tensor(dur),
            con_sequence = conn,
            edg_sequence = edge_attr,
            
            label = torch.tensor(pred_y).float()   
        )

    

class WellDataModule(pl.LightningDataModule):
    
    def __init__(
        self, train_sequences, val_sequences, test_sequences, batch_size=45
    ):
        super().__init__()
        self.train_sequences = train_sequences
        self.val_sequences = val_sequences
        self.test_sequences = test_sequences
        self.batch_size = batch_size
        
    def setup(self):
        self.train_dataset = WellDataset(self.train_sequences)
        self.val_dataset = WellDataset(self.val_sequences)
        self.test_dataset = WellDataset(self.test_sequences)
    def train_dataloader(self):
        return Dataloader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle = False
        )
    def val_dataloader(self):
        return Dataloader(
        self.val_dataset,
        batch_size=1,
        shuffle = False
        )
    def test_dataloader(self):
        return Dataloader(
        self.test_dataset,
        batch_size=1,
        shuffle = False
        )

In [15]:
N_EPOCHS=8
BATCH_SIZE= 46
##We set the batchsize within a single ob well data, which is 81-18-18+1=46. 
data_module = WellDataModule(train_sequences,val_sequences, test_sequences,batch_size =BATCH_SIZE )
data_module.setup()

In [16]:
train_dataset =  WellDataset(train_sequences)
for item in train_dataset:
    print(item.keys())
    print(item['sal_sequence'].shape)
    break

dict_keys(['sal_sequence', 'lev_sequence', 'pum_sequence', 'inf_sequence', 'inf_s_sequence', 'rai_sequence', 'loc_sequence', 'dur_sequence', 'con_sequence', 'edg_sequence', 'label'])
torch.Size([10, 18])


# 2.Model